In [1]:
import os

import autokeras as ak
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.decomposition import PCA

In [2]:
def reduce_mem_usage(df):
    """
    Iterate through all the columns of a dataframe and modify the data type

    Parameters
    ----------
    df : pandas.DataFrame

    Returns
    -------
    pandas.DataFrame

    """

    start_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

In [3]:
# Merge the dataframes on SMILES and CELL_LINE


PATH = "../DrugCell/data_rcellminer/"

t = pd.read_table(PATH + "train_DNA.txt", header=None)
t.columns = ["CELL_LINE", "SMILES", "label"]
t = reduce_mem_usage(t)

pca = PCA(n_components=200)

mfp = pd.read_csv(
    PATH + "drug2fingerprint.csv",
    header=None,
)
mfp = pd.DataFrame(pca.fit_transform(mfp))
mfp["SMILES"] = list(pd.read_csv(PATH + "drug2ind.txt", header=None, sep="\t")[1])
mfp = reduce_mem_usage(mfp)

pca = PCA(n_components=50)

cell_line = pd.read_csv(PATH + "cell2mut.txt", header=None)
cell_line = pd.DataFrame(pca.fit_transform(cell_line))
cell_line["CELL_LINE"] = pd.read_csv(PATH + "cell2ind.txt", sep="\t", header=None)[1]
cell_line = reduce_mem_usage(cell_line)

t = t.merge(mfp, on="SMILES")
train = t.merge(cell_line, on="CELL_LINE")

Memory usage of dataframe is 0.20 MB
Memory usage after optimization is: 0.05 MB
Decreased by 72.9%
Memory usage of dataframe is 0.37 MB
Memory usage after optimization is: 0.10 MB
Decreased by 72.4%
Memory usage of dataframe is 0.02 MB
Memory usage after optimization is: 0.01 MB
Decreased by 63.5%


In [4]:
mfp.shape

(244, 201)

In [5]:
cell_line.shape

(55, 51)

In [6]:
# Define the model


reg = ak.StructuredDataRegressor(overwrite=True, max_trials=100)

In [7]:
# Create val data

t = pd.read_table(
    PATH + 'val_DNA.txt', 
    header=None
    )
t.columns = ['CELL_LINE', 'SMILES', 'label']
t = reduce_mem_usage(t)
t = t.merge(mfp, on='SMILES')    
t = t.merge(cell_line, on='CELL_LINE')

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.03 MB
Decreased by 60.5%


In [8]:
# Fit the model
reg.fit(
    train.drop("label", axis=1),
    train["label"],
    validation_data=(t.drop("label", axis=1), t["label"]),
    epochs=300,
)

Trial 81 Complete [00h 02m 11s]
val_loss: 0.8917642831802368

Best val_loss So Far: 0.7699329853057861
Total elapsed time: 03h 42m 37s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/300
269/269 [==============================] - 5s 11ms/step - loss: 0.7989 - mean_squared_error: 0.7989 - val_loss: 1.1907 - val_mean_squared_error: 1.1907
Epoch 2/300
269/269 [==============================] - 1s 5ms/step - loss: 0.7877 - mean_squared_error: 0.7877 - val_loss: 1.1257 - val_mean_squared_error: 1.1257
Epoch 3/300
269/269 [==============================] - 2s 6ms/step - loss: 0.7775 - mean_squared_error: 0.7775 - val_loss: 1.0799 - val_mean_squared_error: 1.0799
Epoch 4/300
269/269 [==============================] - 2s 6ms/step - loss: 0.7692 - mean_squared_error: 0.7692 - val_loss: 1.0385 - val_mean_squared_error: 1.0385
Epoch 5/300
269/269 [==============================] - 2s 6ms/step - loss: 0.7622 - mean_squared_error: 0.7622 - val_loss: 1.0069 - val_mean_squared_error: 1.0069
Epoch 6/300
269/269 [==============================] - 2s 6ms/step - loss: 0.7565 - mean_squared_error: 0.7565 - val_loss: 0.9820 - val_mean_squared_error: 0.9820
Epoch 7/300
269/269 [

269/269 [==============================] - 2s 8ms/step - loss: 0.6794 - mean_squared_error: 0.6794 - val_loss: 0.7981 - val_mean_squared_error: 0.7981
Epoch 52/300
269/269 [==============================] - 2s 8ms/step - loss: 0.6783 - mean_squared_error: 0.6783 - val_loss: 0.7976 - val_mean_squared_error: 0.7976
Epoch 53/300
269/269 [==============================] - 1s 5ms/step - loss: 0.6773 - mean_squared_error: 0.6773 - val_loss: 0.7972 - val_mean_squared_error: 0.7972
Epoch 54/300
269/269 [==============================] - 1s 5ms/step - loss: 0.6763 - mean_squared_error: 0.6763 - val_loss: 0.7968 - val_mean_squared_error: 0.7968
Epoch 55/300
269/269 [==============================] - 1s 5ms/step - loss: 0.6752 - mean_squared_error: 0.6752 - val_loss: 0.7964 - val_mean_squared_error: 0.7964
Epoch 56/300
269/269 [==============================] - 1s 5ms/step - loss: 0.6741 - mean_squared_error: 0.6741 - val_loss: 0.7961 - val_mean_squared_error: 0.7961
Epoch 57/300
269/269 [=======

269/269 [==============================] - 2s 8ms/step - loss: 0.6305 - mean_squared_error: 0.6305 - val_loss: 0.7869 - val_mean_squared_error: 0.7869
Epoch 102/300
269/269 [==============================] - 1s 5ms/step - loss: 0.6295 - mean_squared_error: 0.6295 - val_loss: 0.7867 - val_mean_squared_error: 0.7867
Epoch 103/300
269/269 [==============================] - 1s 5ms/step - loss: 0.6286 - mean_squared_error: 0.6286 - val_loss: 0.7866 - val_mean_squared_error: 0.7866
Epoch 104/300
269/269 [==============================] - 1s 6ms/step - loss: 0.6278 - mean_squared_error: 0.6278 - val_loss: 0.7866 - val_mean_squared_error: 0.7866
Epoch 105/300
269/269 [==============================] - 1s 5ms/step - loss: 0.6269 - mean_squared_error: 0.6269 - val_loss: 0.7864 - val_mean_squared_error: 0.7864
Epoch 106/300
269/269 [==============================] - 2s 6ms/step - loss: 0.6260 - mean_squared_error: 0.6260 - val_loss: 0.7862 - val_mean_squared_error: 0.7862
Epoch 107/300
269/269 [=

Epoch 151/300
269/269 [==============================] - 2s 9ms/step - loss: 0.5917 - mean_squared_error: 0.5917 - val_loss: 0.7834 - val_mean_squared_error: 0.7834
Epoch 152/300
269/269 [==============================] - 2s 8ms/step - loss: 0.5911 - mean_squared_error: 0.5911 - val_loss: 0.7836 - val_mean_squared_error: 0.7836
Epoch 153/300
269/269 [==============================] - 2s 9ms/step - loss: 0.5905 - mean_squared_error: 0.5905 - val_loss: 0.7832 - val_mean_squared_error: 0.7832
Epoch 154/300
269/269 [==============================] - 2s 8ms/step - loss: 0.5899 - mean_squared_error: 0.5899 - val_loss: 0.7835 - val_mean_squared_error: 0.7835
Epoch 155/300
269/269 [==============================] - 2s 8ms/step - loss: 0.5891 - mean_squared_error: 0.5891 - val_loss: 0.7837 - val_mean_squared_error: 0.7837
Epoch 156/300
269/269 [==============================] - 2s 7ms/step - loss: 0.5885 - mean_squared_error: 0.5885 - val_loss: 0.7836 - val_mean_squared_error: 0.7836
Epoch 157/

269/269 [==============================] - 2s 9ms/step - loss: 0.5647 - mean_squared_error: 0.5647 - val_loss: 0.7874 - val_mean_squared_error: 0.7874
Epoch 201/300
269/269 [==============================] - 2s 9ms/step - loss: 0.5642 - mean_squared_error: 0.5642 - val_loss: 0.7875 - val_mean_squared_error: 0.7875
Epoch 202/300
269/269 [==============================] - 2s 9ms/step - loss: 0.5639 - mean_squared_error: 0.5639 - val_loss: 0.7876 - val_mean_squared_error: 0.7876
Epoch 203/300
269/269 [==============================] - 2s 8ms/step - loss: 0.5634 - mean_squared_error: 0.5634 - val_loss: 0.7878 - val_mean_squared_error: 0.7878
Epoch 204/300
269/269 [==============================] - 2s 9ms/step - loss: 0.5629 - mean_squared_error: 0.5629 - val_loss: 0.7877 - val_mean_squared_error: 0.7877
Epoch 205/300
269/269 [==============================] - 2s 8ms/step - loss: 0.5625 - mean_squared_error: 0.5625 - val_loss: 0.7884 - val_mean_squared_error: 0.7884
Epoch 206/300
269/269 [=

Epoch 250/300
269/269 [==============================] - 2s 9ms/step - loss: 0.5455 - mean_squared_error: 0.5455 - val_loss: 0.7977 - val_mean_squared_error: 0.7977
Epoch 251/300
269/269 [==============================] - 2s 9ms/step - loss: 0.5451 - mean_squared_error: 0.5451 - val_loss: 0.7981 - val_mean_squared_error: 0.7981
Epoch 252/300
269/269 [==============================] - 2s 8ms/step - loss: 0.5448 - mean_squared_error: 0.5448 - val_loss: 0.7980 - val_mean_squared_error: 0.7980
Epoch 253/300
269/269 [==============================] - 2s 8ms/step - loss: 0.5446 - mean_squared_error: 0.5446 - val_loss: 0.7987 - val_mean_squared_error: 0.7987
Epoch 254/300
269/269 [==============================] - 2s 8ms/step - loss: 0.5442 - mean_squared_error: 0.5442 - val_loss: 0.7985 - val_mean_squared_error: 0.7985
Epoch 255/300
269/269 [==============================] - 1s 5ms/step - loss: 0.5440 - mean_squared_error: 0.5440 - val_loss: 0.7989 - val_mean_squared_error: 0.7989
Epoch 256/

269/269 [==============================] - 2s 8ms/step - loss: 0.5311 - mean_squared_error: 0.5311 - val_loss: 0.8112 - val_mean_squared_error: 0.8112
Epoch 300/300
269/269 [==============================] - 2s 8ms/step - loss: 0.5308 - mean_squared_error: 0.5308 - val_loss: 0.8119 - val_mean_squared_error: 0.8119
INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


In [9]:
# Create test data

t = pd.read_table(
    PATH + 'test_DNA.txt', 
    header=None
    )
t.columns = ['CELL_LINE', 'SMILES', 'label']
t = reduce_mem_usage(t)
t = t.merge(mfp, on='SMILES')    
t = t.merge(cell_line, on='CELL_LINE')

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.03 MB
Decreased by 60.5%


In [10]:
predicted_y = reg.predict(t.drop('label', axis=1))

90/90 [==============================] - 1s 8ms/step


In [11]:
spearmanr(pd.DataFrame(predicted_y)[0], t['label'])

SpearmanrResult(correlation=0.4259293781246774, pvalue=8.307437014462078e-127)